In [ ]:
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotter as plotter
import model_manager as model_manager
from IPython.display import display
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.utils import set_random_seed
from custom_env.redes_opticas_env import RedesOpticasEnv
import time

def make_env(num_ont, v_max_olt=10e6,vt_contratada=10e6/10, n_ciclos=200, rank=0, seed=0):
    def _init():
        env = RedesOpticasEnv(render_mode=None,seed=seed, num_ont=num_ont, v_max_olt=v_max_olt, vt_contratada=vt_contratada,n_ciclos=n_ciclos)
        return env
    return _init

if __name__ == "__main__":
    env_id = 'RedesOpticasEnv-v0'  # Hay que asegurarse de que este ID coincida con el registrado
    num_test = 1
    seed = np.random.randint(0, 10)
    num_envs = 1  # Número de entornos paralelos
    num_ont=16
    v_max_olt=10e9 # 10 Gpbs (XGSPON) 
    T=0.002
    OLT_Capacity=v_max_olt*T
    vt_contratada=600e6
    Max_bits_ONT=vt_contratada*T
    algorithm = "PPO"

    n_ciclos=int(input("Cuantos ciclos quiere ver: "))

    vec_env = DummyVecEnv([make_env(num_ont, v_max_olt,vt_contratada,n_ciclos, rank=i, seed=42) for i in range(num_envs)])
    model = model_manager.create_model(vec_env, algorithm)

    start_time = time.time()
    model.learn(total_timesteps=1000)
    end_time = time.time()
    training_time = end_time - start_time
    print(f"El tiempo de entrenamiento fue de {training_time} segundos.")


    # Fase de pruebas
    episode_info = []  
    list_ont = []
    list_ont_2 = []
    list_pendiente=[]
    estados_on_off_recolectados = []
    tamano_cola=[] # Capacidad de la OLT

    obs = vec_env.reset()
    _states = None
    for episode in range(num_test):
        
        done = np.array([False]*num_envs)
        step_counter = 0

        while step_counter < n_ciclos:

            action, _states = model.predict(obs, state=_states, deterministic=True)
            obs, rewards, dones, info = vec_env.step(action)

            episode_info.append(info)
            for i in range(len(info)):  # Itera sobre cada sub-entorno
                suma = 0
                list_ont.append(info[i]['trafico_entrada'])
                list_ont_2.append(info[i]['trafico_salida'])
                list_pendiente.append(info[i]['trafico_pendiente'])
                estados_on_off_recolectados.append(info[i]['trafico_IN_ON_actual'])
            
            done |= dones  # Actualiza 'done' para todos los entornos
            step_counter += 1


    # Graficas
    trafico_entrada = plotter.process_traffic(list_ont, T)
    trafico_salida = plotter.process_traffic(list_ont_2, T)
    trafico_pendiente = plotter.process_traffic(list_pendiente, T)
    valores_instantes = plotter.calculate_instants(estados_on_off_recolectados, num_ont)

    for i in range(num_ont):
        plotter.plot_input_output(trafico_entrada[i], trafico_salida[i], i)
        plotter.plot_pareto(valores_instantes[i], i, n_ciclos)
        plotter.plot_pending(trafico_pendiente[i], i)